In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Dropout
from tensorflow.keras.callbacks import EarlyStopping
InputNo=10

def generate_data(num_samples):
    x = np.random.randint(0, 10, (num_samples, InputNo))
    y = np.flip(x, axis=1)
    return x, y

num_samples = 20000
x, y = generate_data(num_samples)
x = x.reshape((x.shape[0], x.shape[1], 1))
y = y.reshape((y.shape[0], y.shape[1], 1))

model = Sequential()
model.add(LSTM(128, input_shape = (InputNo, 1))) #시퀀스의 각 타임스텝에서 128차원 출력 벡처 생성
model.add(Dropout(0.2))
model.add(RepeatVector(InputNo)) #128차원 출력 벡터를 10번 복제하여 시퀀스 길이 10으로 설정
model.add(LSTM(128, return_sequences=True)) #
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(10, activation='softmax')))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience= 5, restore_best_weights=True)

model.fit(x, y, epochs=50, validation_split=0.2, callbacks=[early_stopping])

def predict_reverse(input_sequence):
    input_sequence=np.array(input_sequence).reshape((1, InputNo, 1))
    predicted_sequence=model.predict(input_sequence)
    return np.argmax(predictedsequence, axis=-1).reshape((InputNo, ))

while True:
    user_input = input(f"입력 (쉼표로 구분된 {InputNo} 개의 숫자, 종료하려면 'quit' 입력): ")
    if user_input.lower()=='quit':
        break
    try:
        input_sequence = list(map(int, user_input.split(',')))
        if len(input_sequence) != InputNo:
            print(InputNo, "개의 숫자를 입력해주세요.")
            continue
        predicted_output = predict_reverse(input_sequence)
        print(f"입력: {input_sequence}")
        print(f"예측된 출력: {predicted_output}")
    except ValueError:
        print("유효한 숫자를 입력해 주세요")

x_test, y_test = generaste_data(20)

correct_pred = 0
total_pred = len(x_test)

for i in range(total_pred):
    input_sequence = x_test[i]
    true_output = y_test[i].reshape(InputNo, )
    predicted_output = predict_reverse(input_sequence)

    if np.array_equal(predicted_output, true_output):
        correct_predictions += 1

    print(f"입력: {input_sequence.reshape(InputNo,)}")
    print(f"예측된 출력: {predicted_output}")
    print(f"실제 출력: { correct_pred}")
    print(f"정확 여부: {'맞음' if np.array_equal(predicted_output, true_output) else '틀림'}\n")

accuracy = correct_predictions / total_predictions
print(f"총 정확도: {accuracy * 100:.2f}%")
        

Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 12s 18ms/step - accuracy: 0.1424 - loss: 2.2387 - val_accuracy: 0.2092 - val_loss: 2.0476
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.2234 - loss: 2.0086 - val_accuracy: 0.2866 - val_loss: 1.8084
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.2939 - loss: 1.7791 - val_accuracy: 0.4033 - val_loss: 1.4835
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.3900 - loss: 1.5008 - val_accuracy: 0.5482 - val_loss: 1.1756
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.5026 - loss: 1.2265 - val_accuracy: 0.7033 - val_loss: 0.9067
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.5853 - loss: 1.0277 - val_accuracy: 0.7957 - val_loss: 0.7358
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.6416 - loss: 0.8972 - val_accuracy: 0.8178 - val_loss: 0.6500
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.6712 - loss: 0.8197 - 

Seq2Seq 인코더 디코더 구분 구현

In [17]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping

InputNo=10

def generate_data(num_samples):
    x = np.random.randint(0, 10, (num_samples, InputNo))
    y = np.flip(x, axis=1)
    return x, y

num_samples = 20000
x, y = generate_data(num_samples)
x = x.reshape((x.shape[0], x.shape[1], 1))
y = y.reshape((y.shape[0], y.shape[1], 1))

#encoder
encoder_inputs = Input(shape=(InputNo, 1))
encoder = LSTM(128, return_state=True)
encoder_ouputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

#decoder
decoder_inputs = Input(shape=(InputNo, 1))
decoder = LSTM(128, return_state=True, return_sequences = True)
decoder_outputs, _, _ = decoder(decoder_inputs, initial_state = encoder_states)
decoder_dense=Dense(10, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

decoder_input_data = np.zeros_like(x)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights = True)

model.fit([x, decoder_input_data], y, epochs=50, validation_split=0.2, callbacks=[early_stopping])

def predict_reverse(input_sequence):
    input_sequence=np.array(input_sequence).reshape((1, InputNo, 1))
    decoder_input = np.zeros((1, InputNo, 1))
    predicted_sequence=model.predict([input_sequence, decoder_input])
    return np.argmax(predicted_sequence, axis=-1).reshape((InputNo, ))


x_test, y_test = generate_data(20)

correct_pred = 0
total_pred = len(x_test)

for i in range(total_pred):
    input_sequence = x_test[i]
    true_output = y_test[i].reshape(InputNo, )
    predicted_output = predict_reverse(input_sequence)

    if np.array_equal(predicted_output, true_output):
        correct_pred += 1

    print(f"입력: {input_sequence.reshape(InputNo,)}")
    print(f"예측된 출력: {predicted_output}")
    print(f"실제 출력: {correct_pred}")
    print(f"정확 여부: {'맞음' if np.array_equal(predicted_output, true_output) else '틀림'}\n")

accuracy = correct_pred / total_predictions
print(f"총 정확도: {accuracy * 100:.2f}%")
        


Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.1742 - loss: 2.1396 - val_accuracy: 0.4490 - val_loss: 1.4475
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step - accuracy: 0.5502 - loss: 1.2176 - val_accuracy: 0.8007 - val_loss: 0.7573
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.8451 - loss: 0.6683 - val_accuracy: 0.9226 - val_loss: 0.4477
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - accuracy: 0.9302 - loss: 0.4022 - val_accuracy: 0.9711 - val_loss: 0.2522
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.9706 - loss: 0.2287 - val_accuracy: 0.9823 - val_loss: 0.1526
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.9722 - loss: 0.1602 - val_accuracy: 0.9673 - val_loss: 0.1386
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.9843 - loss: 0.1000 - val_accuracy: 0.9945 - val_loss: 0.0598
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - accuracy: 0.9804 - loss: 0.0899 - val_acc

NameError: name 'correct_predictions' is not defined

In [28]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping


InputNo = 5 #10으로 하면 (Seq2Seq 방식) 정확도가 떨어짐 LSTM 이라
chars = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'
num_classes = len(chars)

char_to_index = {char: idx for idx, char in enumerate(chars)}
index_to_char = {idx: char for idx, char in enumerate(chars)}

train_data = np.random.choice (list(chars), size = (10000, InputNo))
label_data = np.flip(train_data, axis=1)

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

train_data_indices = np.array([[char_to_index[char] for char in seq] for seq in train_data])
label_data_indices = np.array([[char_to_index[char] for char in seq] for seq in label_data])

print(train_data[:4])
print(label_data[:4])

padded_train = pad_sequences(train_data_indices, maxlen=InputNo)
padded_labels = pad_sequences(label_data_indices, maxlen=InputNo)
print(padded_train[0])


#encoder
encoder_inputs = Input(shape=(InputNo, 1))
encoder = LSTM(128, return_state=True)
encoder_ouputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

#decoder
decoder_inputs = Input(shape=(InputNo, 1))
decoder = LSTM(128, return_state=True, return_sequences = True)
decoder_outputs, _, _ = decoder(decoder_inputs, initial_state = encoder_states)
decoder_dense=Dense(52, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

decoder_input_data = np.zeros_like(padded_train)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights = True)

model.fit([padded_train, decoder_input_data], padded_labels, epochs=100, validation_split=0.2, callbacks=[early_stopping])

[['Z' 'x' 'e' 'n' 'l']
 ['m' 'z' 'X' 'N' 'w']
 ['p' 'i' 'S' 'T' 't']
 ['Q' 'm' 'n' 'w' 'T']]
[['l' 'n' 'e' 'x' 'Z']
 ['w' 'N' 'X' 'z' 'm']
 ['t' 'T' 'S' 'i' 'p']
 ['T' 'w' 'n' 'm' 'Q']]
[25 49 30 39 37]
Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.0320 - loss: 3.8452 - val_accuracy: 0.0685 - val_loss: 3.3770
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.0939 - loss: 3.1589 - val_accuracy: 0.1555 - val_loss: 2.6330
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.1834 - loss: 2.5186 - val_accuracy: 0.2230 - val_loss: 2.2907
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.2583 - loss: 2.2041 - val_accuracy: 0.2847 - val_loss: 2.0444
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.3248 - loss: 1.9847 - val_accuracy: 0.3444 - val_loss: 1.8668
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.3872 - loss: 1.8098 - val_accuracy: 0.3744 - val_loss: 1.7404
Epoch 7/100
250/250 ━━

In [46]:
def predict_reverse(input_sequence):
    input_sequence=np.array(input_sequence).reshape((1, InputNo, 1))
    decoder_input = np.zeros((1, InputNo, 1))
    predicted_sequence=model.predict([input_sequence, decoder_input])
    return np.argmax(predicted_sequence, axis=-1).reshape((InputNo, ))


test_data = ["".join(np.random.choice(list(chars), InputNo)) for _ in range(20)]
expected_outputs = [s[::-1] for s in test_data]

print (test_data)
correct_pred = 0
total_pred = len(test_data)

for i, test_string in enumerate(test_data):
    numbers = [char_to_index[char] for char in test_string]
    array_numbers = np.array([numbers])
    padded_numbers = pad_sequences(array_numbers, maxlen=InputNo)
    padded_numbers = padded_numbers.reshape(1, InputNo, 1)

    decoder_input = np.zeros((1, InputNo, 1))  # ADD THIS LINE

    prediction = model.predict([padded_numbers, decoder_input])  # FIXED HERE
    predicted_indices = np.argmax(prediction, axis=-1)[0]
    predicted_chars = [index_to_char[idx] for idx in predicted_indices]
    predicted_output = "".join(predicted_chars)

    is_correct = predicted_output == expected_outputs[i]
    if is_correct:
        correct_pred += 1
    print(f" Input: {test_string}")
    print(f" Est. Output: {predicted_output}")
    print(f" Expected Output: {expected_outputs[i]}")
    print(f" Correct: {'Yes' if is_correct else 'Wrong'}\n")


accuracy = correct_pred / total_pred
print(f" Total Acc: {accuracy*100:.2f}%")

['KPvzQ', 'TwLjb', 'JuShn', 'ROcgH', 'BBKtW', 'UIGOv', 'VotVB', 'krjcL', 'NlgEA', 'zgrAT', 'rJyRz', 'kuGqo', 'ltwho', 'BQUdq', 'PxpNy', 'iQqWq', 'pYGQN', 'QRgAA', 'axyUg', 'zKhYv']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
 Input: KPvzQ
 Est. Output: QzwQK
 Expected Output: QzvPK
 Correct: Wrong

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
 Input: TwLjb
 Est. Output: bjLwT
 Expected Output: bjLwT
 Correct: Yes

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
 Input: JuShn
 Est. Output: nhSuJ
 Expected Output: nhSuJ
 Correct: Yes

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
 Input: ROcgH
 Est. Output: HgcOR
 Expected Output: HgcOR
 Correct: Yes

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
 Input: BBKtW
 Est. Output: WtKBB
 Expected Output: WtKBB
 Correct: Yes

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
 Input: UIGOv
 Est. Output: vOGIU
 Expected Output: vOGIU
 Correct: Yes

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
 Input: VotVB
 Est. Output: BVtoU
 Expected Output: BVtoV
 Correct: Wrong

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
